In [1]:
import os

In [2]:
%pwd

'/Users/main/Desktop/LSTM_Forecast/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/main/Desktop/LSTM_Forecast'

In [5]:
import pandas as pd
import numpy as np
from TimeSeriesForecast import logger
from TimeSeriesForecast.utils.common import read_yaml,create_directories
from TimeSeriesForecast.constants import *
from dataclasses import dataclass
from pathlib import Path
import torch
from notebooks.src.functions_torch import TimeSeries
from notebooks.src.functions_torch import LSTM
# from notebooks.src.functions_torch import set_seed
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    save_path: Path
    model_name: Path
    n_steps: int
    split_ratio: float
    seed: int
    drop_out_rate: float
    input_size: int
    hidden_size: int
    num_stacked_layers: int
    learning_rate: float
    num_epochs: int
    batch_size: int

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH,
                 schema_filepath= SCHEMA_FILE_PATH):
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def model_trainer_config(self)->ModelTrainerConfig:
        config= self.config.model_trainer
        params= self.params.TorchModel
        schema= self.schema
        
        create_directories([config.root_dir])
                
        model_trainer_config= ModelTrainerConfig(
                                                root_dir= config.root_dir,
                                                data_path= config.data_path,
                                                save_path= config.save_path,
                                                model_name= config.model_name,
                                                n_steps= params.n_steps,
                                                split_ratio= params.split_ratio,
                                                seed= params.seed,
                                                drop_out_rate= params.drop_out_rate,
                                                input_size= params.input_size,
                                                hidden_size= params.hidden_size,
                                                num_stacked_layers= params.num_stacked_layers,
                                                learning_rate= params.learning_rate,
                                                num_epochs= params.num_epochs,
                                                batch_size= params.batch_size
                                                )
        return model_trainer_config

In [8]:

class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config= config
        self.tensor= None
        self.X_train= None
        self.X_test= None
        self.y_train= None
        self.y_test= None
    def tensor_loader(self):
        torch.manual_seed(self.config.seed)
        torch.cuda.manual_seed(self.config.seed)
        np.random.seed(self.config.seed)
        try:
            self.tensor= torch.load(self.config.data_path)
            self.X_train,self.X_test,self.y_train,self.y_test = self.tensor['X_train'],self.tensor['X_test'],\
                                                                self.tensor['y_train'],self.tensor['y_test']
            return self
        except Exception as e:
            raise Exception(f'Problem loading the tensors: {e}')
    def time_series_dataset(self):
        try:
            self.train_dataset= TimeSeries(self.X_train,self.y_train)
            self.test_dataset= TimeSeries(self.X_test,self.y_test)
            
            self.train_loader= DataLoader(self.train_dataset,batch_size=self.config.batch_size,shuffle=True)
            self.test_loader= DataLoader(self.test_dataset,batch_size=self.config.batch_size,shuffle=False)
        except Exception as e:
            raise Exception(f'Problem the class modules {e}')
        return self
    
    def training_torch(self):
        try:
            torch.manual_seed(self.config.seed)
            torch.cuda.manual_seed(self.config.seed)
            np.random.seed(self.config.seed)
            device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
            model= LSTM(self.config.input_size,self.config.hidden_size,self.config.num_stacked_layers)
            model.to(device)
            loss_function = nn.MSELoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=self.config.learning_rate)
            
            for epoch in range(self.config.num_epochs):
                model.train(True)
                running_loss = 0.0

                for _, batch in enumerate(self.train_loader):
                    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

                    output = model(x_batch)
                    loss = loss_function(output, y_batch)
                    running_loss += loss.item()

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    avg_train_loss= running_loss/len(self.train_loader)
                    torch.save(model,self.config.save_path)
                print(f'Epoch {epoch+1}/{self.config.num_epochs}, Train Loss: {avg_train_loss}')
                logger.info(f'Train loss from Torch Model{avg_train_loss}')
            torch.save(model,self.config.save_path)
        except Exception as e:
            raise Exception(f'Problem training the model {e}')
        

In [9]:
try:
        config= ConfigurationManager()
        model_trainer= config.model_trainer_config()
        model_t= ModelTrainer(config=model_trainer)
        model_t.tensor_loader()
        model_t.time_series_dataset()
        model_t.training_torch()
except Exception as e:
    raise e

[2024-08-06 12:26:23,135:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-08-06 12:26:23,138:INFO:common:yaml file: params.yaml loaded successfully]
[2024-08-06 12:26:23,140:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-08-06 12:26:23,141:INFO:common:created directory at: artifacts]
[2024-08-06 12:26:23,142:INFO:common:created directory at: artifacts/model_trainer]


/Users/main/Desktop/LSTM_Forecast/time/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/10, Train Loss: 0.1992187452163886
[2024-08-06 12:26:23,864:INFO:429443308:Train loss from Torch Model0.1992187452163886]
Epoch 2/10, Train Loss: 0.13584739416414363
[2024-08-06 12:26:23,972:INFO:429443308:Train loss from Torch Model0.13584739416414363]
Epoch 3/10, Train Loss: 0.10072631147605451
[2024-08-06 12:26:24,081:INFO:429443308:Train loss from Torch Model0.10072631147605451]
Epoch 4/10, Train Loss: 0.06948442103616385
[2024-08-06 12:26:24,192:INFO:429443308:Train loss from Torch Model0.06948442103616385]
Epoch 5/10, Train Loss: 0.04388478030027314
[2024-08-06 12:26:24,303:INFO:429443308:Train loss from Torch Model0.04388478030027314]
Epoch 6/10, Train Loss: 0.02234303987841651
[2024-08-06 12:26:24,422:INFO:429443308:Train loss from Torch Model0.02234303987841651]
Epoch 7/10, Train Loss: 0.011270751882958311
[2024-08-06 12:26:24,537:INFO:429443308:Train loss from Torch Model0.011270751882958311]
Epoch 8/10, Train Loss: 0.006724874752850693
[2024-08-06 12:26:24,646:INFO:4